In [182]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
import scipy
from scipy.linalg import solve_toeplitz, toeplitz
%matplotlib tk

In [183]:
## Load data
data, sr = librosa.load('./wav/sa.wav', sr = None)

print(f'data length : {len(data)}, sr : {sr}')


data length : 7988, sr : 16000


In [184]:
win_length  = int(20/1000 * sr)
hop_length = int(win_length / 2)
num_frame = int(len(data) / hop_length) - 1

print(f'win_length = {win_length}, hop_length = {hop_length}, num_frame = {num_frame}')

win_length = 320, hop_length = 160, num_frame = 48


In [185]:
## plot wavefile
plt.figure(figsize=(14,5))
time = np.linspace(0,len(data)/16000, len(data))
plt.plot(time,data)
plt.xlim(0,len(data)/sr)
plt.ylim(-1,1)
plt.grid(True)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Waveform of sa.wav')
plt.rc('font', size=15)


In [186]:
### Note : LPC is obtained from the windowed frame while the residual signal is obtained without windowing

In [187]:
## Data frame화
# 1. window 설정 => Hamming window
window = np.hamming(320)
w = win_length
h = hop_length
num_frames = num_frame

windowed_data = np.zeros((num_frames,w))
for i in range(num_frames):
        windowed_data[i] = data[i * h : i * h + w] * window
print(windowed_data.shape)

(48, 320)


In [188]:
## autocorrelation 함수 설정

def autocorrelation(data, k):
    if k == 0:
        return np.sum(data ** 2)
    else:
        return np.sum(data[k:] * data[:-k])    

In [189]:
## Levinson durbin algorithm

def Levinson_Durbin(frame_signal, p):
    # 각 frame별로 levinson_durbin을 구해보자
    # 즉, 각 frame별로 LPC를 구해보자
    # signal : sampling된 data
    # p : pth order
    # The Toeplitz matrix has constant diagonals, with c as its first column and r as its first row.
    # Right-hand side in T x = b.
    column = row = [autocorrelation(frame_signal,i) for i in range(p)]
    b = [autocorrelation(frame_signal,i) for i in range(1,p+1)] # (1 <=i <= p)이므로
    # Reuturn x
    # x : coefficients a[1],a[2],a[3],...,a[k] 즉 lpc(linear prediction coefficients)
    x = solve_toeplitz((column, row), b)
    return x


In [190]:
print(Levinson_Durbin(windowed_data[3],5))
# Levinson_Durbin(windowded_data[n],p)
# >>> LPC shape : (1,p)

[ 0.2947044   0.93549622  0.08685609 -0.48439713 -0.04616732]


In [191]:
def residual(windowed_data, p):
    
    LPC = Levinson_Durbin(windowed_data,p)
    # >>> LPC shape : (1,p)
    win_length = len(windowed_data)
    # >>> win_length: 320
    prediction = np.zeros((win_length))
    # prediction : 예측치
    # >>> prediction shape : (1, 320)
    
    for k in range(win_length):
        for j in LPC:
            prediction[k] += j * windowed_data[k]
    
        
    error = windowed_data - prediction
    return prediction, error

In [192]:
def predict(data, windowed_data, win_length, p):
    W = win_length
    H = int(win_length / 2)
    nF = int(len(data) / hop_length) - 1
    
    # 프레임 => 전체
    total_pred = np.zeros((len(data)))
    total_err =  np.zeros((len(data)))
   
    for n in range(nF):
        
        predict, error = residual(windowed_data[n],p)
        
        total_pred[n * H : n * H + W] += predict
        total_err[n * H : n * H + W] += error
        
    return total_pred, total_err 

In [209]:
## plot the LPC
# LPC order = 32
time_axis = np.linspace(0,len(data) / sr, len(data))
total_pred, total_err = predict(data, windowed_data, win_length, 32)

In [210]:
plt.figure(figsize = (14,5))
plt.plot(time_axis, data, label = 'Original waveform', color = 'g') # Original data

plt.plot(time_axis, total_pred, '--', color = 'r', label = 'Prediction waveform') 
plt.xlim(0.2,0.3)
plt.ylim(-1,1.1)
plt.grid(True)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('<Original waveform and Prediction waveform>')
plt.legend()
plt.rc('font', size=15)

In [211]:
## plot error
plt.figure(figsize = (14,5))
# plt.plot(time_axis, data, label = 'Original waveform', color = 'g') # Original data

plt.plot(time_axis, total_err, color = 'r', label = 'error waveform') 
plt.xlim(0.2,0.3)
plt.ylim(-0.1,0.1)
plt.grid(True)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Error waveform ')
plt.legend()
plt.rc('font', size=15)

In [214]:
def TF_spectrum(signal, window, sr, p):
    
    plt.figure(figsize=(14,5))
    win_sig = signal*window
    LPC = Levinson_Durbin(win_sig, p) # LPC 계산, shape : (1,32)
    #LPC_negative = np.ones((p)) - LPC
    LPC_negative = np.insert(-LPC, 0, 1) # 분모에서 1-LPC임
    print(LPC_negative)
    G_square = np.array(autocorrelation(win_sig,0)) - np.sum(LPC * [autocorrelation(win_sig, i) for i in range(1,p+1)]) # Voiced Gain
    G = G_square ** (1/2) # Voiced Gain
    print(f'G : {G}')
    axis = np.linspace(0,sr/2,1024)
    _, h2 = scipy.signal.freqz(G, LPC_negative, worN=1024, fs = sr) # frequency response 계산 패키지
        
    fft_signal = np.fft.rfft(win_sig,2048)
    plt.plot(axis,20*np.log(abs(fft_signal[1:])),label = 'Signal spectrum')  
    plt.plot(axis,20*np.log(abs(h2)),label = 'Frequency response')
    plt.xlim(0,sr/2)
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Magnitude [dB]')
    plt.title('Spectrum of transfer function, order: {}'.format(p))
    plt.legend()
    plt.grid(True)
    plt.rc('font', size=15)
    plt.show()

In [215]:
# Unvoiced

TF_spectrum(data[2600:2920], np.hamming(320),sr,32)

[ 1.         -1.19112077  0.24872118 -0.37458158  0.74612534 -0.6039218
  0.70518775 -0.42578614  0.34307818 -0.43350365  0.36048256 -0.60038652
  0.27108208 -0.02357374  0.52842301 -0.39281625  0.20868371 -0.33640164
  0.06819228 -0.35285473  0.61196314 -0.39611643  0.37287882 -0.37892975
  0.39028345 -0.42635105  0.29862489 -0.3368354   0.210684   -0.00733479
  0.1802712  -0.30784302  0.10289315]
G : 0.16111252472591203


In [198]:
# voiced
TF_spectrum(data[3520:3840], np.hamming(320),sr,32)

[ 1.         -2.62897716  2.27156133 -0.47864984 -0.28005451  0.37546316
 -0.30775166 -0.06044603  0.23859587 -0.11481829 -0.06634529 -0.11806366
  0.48405099 -0.52840015  0.58003121 -0.54238199  0.17884317  0.02287139
 -0.10865381  0.18283037 -0.00597756 -0.14430679  0.11445437 -0.16597672
  0.29970156 -0.32700959  0.15825248 -0.07892467  0.23342734 -0.16777219
 -0.09690993  0.12565699 -0.02386404]
G : 0.09457373203575571
